In [32]:
import cifar10
import tensorflow as tf

In [33]:
cifar10.data_path = "data/CIFAR-10/"
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [34]:
class_names = cifar10.load_class_names()

Loading data: data/CIFAR-10/cifar-10-batches-py/batches.meta


In [35]:
x_train, y_train, labels = cifar10.load_training_data()

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5


In [45]:
x_test, y_test, labels_test = cifar10.load_test_data()

AttributeError: module 'cifar10' has no attribute 'load_testing_data'

In [37]:
input_width = 32
input_height = 32
input_channels = 3
n_input = 3072
n_conv1 = 32
n_conv2 = 64
conv1_k = 5
conv2_k = 5
n_hidden = 1024
n_out = 10
pooling_window_size = 2

In [38]:
weights = {
    "wc1" : tf.Variable(tf.random_normal([conv1_k, conv1_k ,input_channels, n_conv1])),
    "wc2" : tf.Variable(tf.random_normal([conv2_k, conv2_k ,n_conv1, n_conv2])),
    "wh1" : tf.Variable(tf.random_normal([input_width//(pooling_window_size*pooling_window_size) * input_height//(pooling_window_size*pooling_window_size) * n_conv2, n_hidden])),
    "wout": tf.Variable(tf.random_normal([n_hidden, n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random_normal([n_conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv2])),
    "bh1" : tf.Variable(tf.random_normal([n_hidden])),
    "bout": tf.Variable(tf.random_normal([n_out]))
}

In [39]:
def conv(x, weights, bias, strides = 1):
    out = tf.nn.conv2d(x, weights, padding="SAME", strides=[1, strides, strides, 1])
    out = tf.nn.bias_add(out, bias)
    return tf.nn.relu(out)

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding="SAME", ksize=[1, k, k, 1], strides=[1, k, k, 1])

In [40]:
def cnn(x, weights, biases):
    x = tf.reshape(x, shape = [-1,input_height, input_width, 3])
    conv1 = conv(x, weights["wc1"], biases["bc1"])
    conv1 = maxpooling(conv1, k = pooling_window_size)
    
    conv2 = conv(conv1, weights["wc2"], biases["bc2"])
    conv2 = maxpooling(conv2, k = pooling_window_size)
    
    hidden_input = tf.reshape(conv2, shape=[-1, input_width//4 * input_height//4 * n_conv2])
    hidden_output = tf.add(tf.matmul(hidden_input, weights["wh1"]), biases['bh1'])
    hidden_output = tf.nn.relu(hidden_output)
    
    out =  tf.add(tf.matmul(hidden_output, weights["wout"]), biases['bout'])
    return out

In [41]:
x = tf.placeholder("float", [None, 32,32,3])
y = tf.placeholder("float", [None, n_out])
pred = cnn(x, weights, biases)

In [42]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimize = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [43]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [44]:
batch_size = 100
for i in range(25):
    num_batches = int(x_train.shape[0]/batch_size)   
    total_cost = 0
    for j in range(num_batches):
        batch_x, batch_y = x_train[j*batch_size:(j+1)*batch_size],labels[j*batch_size:(j+1)*batch_size]
    
        #print(j*batch_size, " ", (j+1)*batch_size)
        _ , c = sess.run([optimize, cost], feed_dict={x:batch_x, y:batch_y})
        print(c)
        total_cost += c
    print("************COST**************  ",total_cost)

119237.0
204340.0
213981.0
208543.0
198679.0
218212.0
140636.0
128759.0
140062.0
73615.4
61669.9
37903.2
31470.4
21586.0
18840.2
15767.8
15033.9
12771.1
8203.46
8607.41
9407.82
6063.15
7945.07
6909.29
5828.61
5062.56
4056.45
3434.17
2813.34
3956.49
3283.07
3690.84
3438.87
2909.1
2424.53
2437.08
2243.95
2142.15
1825.37
1680.42
1641.49
1589.58
1064.21
1360.66
1427.84
1362.33
1257.94
964.307
1039.92
1015.73
985.243
972.428
942.813
683.354
772.487
767.933
830.632
730.578
603.656
617.101
535.199
626.325
638.313
588.672
666.822
491.719
475.216
430.45
476.856
380.619
445.224
421.683
508.462
412.472
412.579
364.021
387.204
353.537
285.138
335.099
307.535
335.747
217.74
261.246
290.23
260.907
309.348
244.483
245.62
243.809
221.204
166.844
204.13
163.904
226.098
149.657
126.634
225.175
200.339
117.182
172.053
200.365
107.04
185.947
154.14
161.446
129.932
156.374
142.163
105.174
159.245
120.612
154.212
148.597
129.912
108.53
104.863
181.552
108.745
112.695
142.657
85.5975
116.088
107.162
70.3823


7.46146
3.47521
2.40036
2.71315
2.39654
3.47538
6.17785
3.47399
3.82138
3.22397
2.36773
2.50172
2.34448
4.78819
3.86995
2.29886
3.24107
2.35246
2.3629
2.90473
2.76866
2.85263
4.20201
3.20476
2.44121
3.10682
2.31238
2.32607
2.89254
2.5825
2.51956
3.18648
2.37867
2.72983
2.41012
2.47506
2.47332
3.2074
2.32728
2.37721
2.37184
2.75679
2.38417
2.60597
2.3452
2.44637
3.25778
3.20968
2.44127
3.21035
3.27738
2.3527
2.35817
2.55307
2.70089
2.7874
2.50109
5.77605
2.52981
3.04045
2.39222
3.05968
2.58348
2.51552
2.53548
3.15132
3.4088
2.87329
2.49351
2.68643
2.91321
3.83356
2.51324
2.35436
2.30634
2.40206
2.37108
4.04347
3.02715
2.32883
2.84369
2.37485
2.6934
4.42734
2.33601
2.40644
2.65937
2.94203
2.5874
3.10842
3.70697
2.60426
2.40597
5.57194
3.10253
2.8157
2.37469
2.64046
2.43416
2.28479
2.33804
3.33826
2.57048
2.55443
2.82483
4.70113
4.8939
3.75768
2.45424
2.37202
6.12182
3.22731
2.62922
2.34187
2.37539
3.04364
2.25378
2.37022
2.32197
2.39487
3.85879
2.93473
2.32336
2.66691
2.42852
2.34603
2.6

2.32936
2.37818
2.41841
2.35812
2.35982
2.35609
2.32064
2.354
2.38059
2.32299
2.36171
2.27734
2.33179
2.3656
2.37041
2.43296
2.37516
2.36634
2.36496
2.34833
2.3711
3.28452
2.35394
2.48189
2.45197
2.30555
2.40576
2.44188
2.7345
2.39011
2.46403
3.14805
2.55568
2.54134
2.38331
2.36004
2.44143
2.51832
2.45975
2.48692
2.49442
2.57585
2.38448
2.3679
2.5295
2.44439
2.5085
2.32817
2.3264
2.40714
2.38566
2.52793
2.39576
2.3232
2.67294
2.39218
2.52422
3.07287
2.32178
2.2775
2.2878
2.38779
2.56871
2.4648
2.25067
2.35306
2.39636
2.57357
2.30144
2.39547
2.38253
2.43502
2.44143
2.5351
2.32241
3.06834
2.38663
2.43225
2.36435
2.33562
2.73046
2.43908
2.31194
2.40199
2.36342
2.42743
2.3931
2.30938
2.50636
2.3994
2.30633
2.33027
2.3424
2.37097
2.99436
2.30807
2.34467
2.37717
2.40411
2.33776
2.35777
2.40851
2.40933
2.75023
2.39472
2.88233
3.29
2.36503
2.41855
2.40902
2.29323
2.38443
2.71704
2.36854
2.69719
2.55551
2.2824
2.37403
2.53039
2.34585
2.59443
2.33113
2.43567
2.41688
2.40122
2.35108
2.5202
2.3258

2.40217
2.37396
2.52746
2.50665
2.60542
2.63347
2.35516
2.37767
2.43592
2.52259
2.42279
2.60249
2.45997
2.63791
2.3873
2.38698
2.46618
2.4336
2.5332
2.33944
2.33119
2.417
2.4023
2.48232
2.38741
2.2996
2.39922
2.4273
2.54448
2.44873
2.32805
2.31119
2.29666
2.40974
2.51533
2.36954
2.26818
2.36252
2.41642
2.45061
2.30538
2.35431
2.38843
2.43405
2.43008
2.30708
2.32101
3.1679
2.40019
2.42683
2.3739
2.33992
2.46217
2.35504
2.32051
2.33509
2.37454
2.44592
2.70566
2.30526
2.37015
2.42517
2.3127
2.32445
2.33763
2.36127
2.32976
2.3179
2.34246
2.38145
2.42048
2.35899
2.36001
2.42935
2.39624
2.49125
2.40172
2.59137
2.58177
2.35229
2.49365
2.40446
2.29023
2.41267
2.3331
2.37101
2.43558
2.35718
2.94599
2.37013
2.34746
2.33358
2.326
2.34151
2.43647
2.37678
2.40101
2.36336
2.55797
2.39397
2.3581
2.38199
2.38942
2.41333
2.41008
2.35144
2.4267
2.37588
2.34233
2.35377
2.40834
2.39205
2.32054
2.39138
2.39563
2.45089
2.32985
2.34204
2.45286
2.36238
2.41296
2.3778
2.36357
2.37148
2.30154
2.39769
2.35487
2.

2.44584
2.5243
2.28097
2.34139
2.33189
2.30515
2.40036
2.51029
2.35108
2.29616
2.36218
2.41476
2.54417
2.29765
2.35593
2.3855
2.40733
2.39239
2.2994
2.33033
2.44605
2.39673
2.39607
2.36769
2.37429
2.40827
2.38023
2.32424
2.29118
2.37514
2.44098
2.40359
2.30879
2.39437
2.45301
2.32003
2.31951
2.35434
2.35532
2.3431
2.31712
2.34291
2.38838
2.41499
2.35213
2.32889
2.45667
2.3866
2.46159
2.38338
2.61173
2.40727
2.37883
2.49767
2.39141
2.29606
2.44653
2.33042
2.38189
2.39503
2.37152
2.3208
2.36676
2.3649
2.35464
2.31986
2.33149
2.40845
2.35605
2.39377
2.37738
2.59859
2.31778
2.34521
2.35988
2.37248
2.42717
2.41364
2.33898
2.44184
2.35812
2.32697
2.36142
2.42475
2.37232
2.32994
2.40658
2.39675
2.46972
2.33013
2.34818
2.45882
2.36699
2.44511
2.35435
2.37806
2.4063
2.31077
2.40518
2.34596
2.33463
2.37224
2.39982
2.42366
2.38769
2.34699
2.33657
2.39173
2.38503
2.51362
2.40023
2.35958
2.29946
2.48405
2.73166
2.22913
2.48201
2.34973
2.36329
2.43869
2.40939
2.44983
2.37282
2.35276
2.46426
2.33775


...

2.34401
2.34603
2.31647
2.37372
2.4899
2.3273
2.30514
2.36117
2.56244
2.31534
2.29997
2.42396
2.37954
2.36917
2.37743
2.35727
2.44063
2.33929
2.42717
2.45429
2.33216
2.39731
2.44914
2.4519
2.36348
2.49452
2.507
2.51314
2.41463
2.61332
2.67237
2.3953
2.39889
2.45642
2.46172
2.43074
2.41022
2.45136
2.49867
2.46999
2.48412
2.6467
2.4175
2.47123
2.40403
2.40916
2.44004
2.4409
2.35728
2.29302
2.47494
2.43937
2.44094
2.36587
2.40241
2.30952
2.35676
2.45503
2.39221
2.36505
2.36038
2.33254
2.3824
2.37266
2.46867
2.31372
2.29979
2.52363
2.45703
2.36605
2.45048
2.58733
2.37653
2.34425
2.41373
2.4846
2.39927
2.33325
2.45176
2.47641
2.54234
2.5293
2.44854
2.47769
2.38567
2.54928
2.41216
2.42705
2.38912
2.38309
2.42614
2.50425
2.32981
2.35608
2.50215
2.34626
2.40938
2.35464
2.38006
2.34795
2.34823
2.49511
2.46583
2.52698
2.37402
2.39819
2.42272
2.40688
2.40695
2.34972
2.35788
2.34563
2.38082
2.39917
2.3895
2.26215
2.39948
2.6277
2.44089
2.35639
2.40343
2.6856
2.46272
2.42859
2.35299
2.54571
2.51141

2.42071
2.67075
2.71374
2.38396
2.4146
2.48827
2.48485
2.43026
2.40757
2.4346
2.51881
2.48652
2.50542
2.65038
2.4307
2.48091
2.41904
2.4479
2.46625
2.43761
2.35866
2.2859
2.49658
2.43401
2.41434
2.34345
2.42816
2.33284
2.35822
2.43904
2.3736
2.38029
2.36894
2.34241
2.405
2.34498
2.44477
2.31266
2.30139
2.53845
2.43945
2.33358
2.46815
2.56952
2.36986
2.37727
2.40731
2.48634
2.40079
2.33813
2.4402
2.44086
2.51919
2.5405
2.44357
2.44596
2.40577
2.57602
2.41575
2.42534
2.39412
2.39427
2.42285
2.52633
2.32375
2.39506
2.54295
2.34125
2.41849
2.38403
2.37313
2.35725
2.32664
2.47197
2.49236
2.53443
2.37143
2.40881
2.43345
2.44771
2.42581
2.33836
2.36691
2.34016
2.42326
2.40604
2.36367
2.26015
2.44217
2.62653
2.44522
2.368
2.41056
2.71468
2.46751
2.42699
2.36467
2.57758
2.5367
2.54326
2.38842
2.48625
2.54653
2.4892
2.49793
2.35889
2.57511
2.35346
2.4291
2.41319
2.50671
2.3731
2.35366
2.40887
2.42764
2.42988
2.42866
2.48048
2.43873
2.34849
2.38856
2.46905
2.37922
2.35293
2.50229
2.38429
2.31888


2.34517
2.36078
2.41848
2.36081
2.39444
2.38311
2.34122
2.42121
2.32694
2.43278
2.31862
2.31074
2.51963
2.4331
2.32979
2.48296
2.53052
2.37531
2.41327
2.41177
2.48901
2.40754
2.34353
2.44155
2.41754
2.49574
2.56179
2.41741
2.43547
2.41743
2.58458
2.41207
2.42742
2.38395
2.40603
2.4327
2.52025
2.32293
2.43134
2.56624
2.33816
2.42511
2.41127
2.37913
2.36481
2.30458
2.45506
2.51077
2.52499
2.36151
2.40756
2.45435
2.47541
2.42404
2.32981
2.37047
2.34154
2.46356
2.38924
2.3446
2.27053
2.46689
2.61087
2.44817
2.37591
2.41365
2.72425
2.44588
2.42106
2.37799
2.57672
2.53986
2.55279
2.40037
2.50984
2.54682
2.50004
2.52407
2.38785
2.5902
2.36348
2.39822
2.40821
2.52012
2.37386
2.35079
2.39703
2.41815
2.44117
2.4242
2.47817
2.42648
2.38196
2.39675
2.46793
2.39999
2.3583
2.50437
2.39887
2.32125
2.39528
2.43745
2.55283
2.42874
2.39486
2.49433
2.47349
2.38532
2.47352
2.48038
2.46354
2.39062
2.49422
2.61036
2.43783
2.51526
2.35224
2.47342
2.52579
2.44067
2.42136
2.41325
2.44267
2.31646
2.38648
2.3722

2.40504
2.4922
2.57454
2.3891
2.44139
2.42519
2.58751
2.41039
2.42759
2.36832
2.42599
2.43935
2.50152
2.33063
2.45937
2.58044
2.33615
2.42913
2.42829
2.39245
2.36695
2.29017
2.4536
2.52825
2.51216
2.35262
2.41172
2.47523
2.48238
2.41357
2.32971
2.36945
2.34435
2.4827
2.36632
2.33818
2.28035
2.47629
2.59566
2.44451
2.37432
2.42072
2.72467
2.41859
2.42056
2.38156
2.55869
2.53845
2.54716
2.40328
2.50967
2.54356
2.51328
2.5373
2.40375
2.58328
2.37766
2.37375
2.41274
2.52163
2.37506
2.35048
2.39276
2.40938
2.44457
2.41528
2.46982
2.41752
2.40545
2.39538
2.46061
2.41661
2.36223
2.5019
2.40408
2.32754
2.38281
2.43693
2.57264
2.42276
2.42461
2.49601
2.4751
2.39318
2.47025
2.48907
2.46614
2.39849
2.5127
2.62175
2.4448
2.5017
2.34761
2.49862
2.52921
2.43403
2.41398
2.40151
2.45551
2.31572
2.38145
2.36095
2.39054
2.35011
2.35257
2.42086
2.28562
2.4385
2.29984
2.40158
2.36795
2.4968
2.38642
2.3859
2.37052
2.38589
2.34182
2.49256
2.42949
2.34017
2.36791
2.53016
2.38058
2.32247
2.37089
2.46324
2.348

2.48507
2.47912
2.40709
2.33503
2.36568
2.34471
2.48691
2.35316
2.34218
2.27859
2.48199
2.58253
2.43743
2.37025
2.43195
2.71906
2.39609
2.42452
2.37639
2.53917
2.53275
2.53928
2.40537
2.5016
2.53291
2.52537
2.54433
2.4105
2.57028
2.38616
2.35606
2.42575
2.5234
2.37253
2.35147
2.3976
2.40736
2.44261
2.40787
2.46536
2.40875
2.41017
2.39128
2.45855
2.41668
2.36256
2.50125
2.39932
2.32703
2.37565
2.44014
2.57911
2.41098
2.43757
2.49855
2.4805
2.39795
2.46134
2.50109
2.47179
2.40267
2.50818
2.63463
2.46047
2.49452
2.34698
2.51921
2.53059
2.43205
2.41263
2.39796
2.45755
2.31393
2.38263
2.35998
2.37981
2.35963
2.35441
2.41245
2.2859
2.44021
2.30423
2.3986
2.36718
2.49959
2.38057
2.39151
2.37019
2.38482
2.3439
2.49525
2.42671
2.33555
2.37872
2.53658
2.37726
2.32352
2.38193
2.46882
2.34134
2.43958
2.40927
2.4083
2.44101
2.45393
2.42012
2.43018
2.43418
2.39247
2.42711
2.42658
2.37882
2.40064
2.4788
2.34977
2.61786
2.34252
2.36045
2.26532
2.548
2.40125
2.42602
2.46063
2.45391
2.47162
2.34265
2.34

In [ ]:
predictions = tf.argmax(pred, 1)
correct_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, correct_labels)
predictions,correct_predictions  = sess.run([predictions, correct_predictions], feed_dict={x:x_test,
                                              y:labels_test})


In [ ]:
(correct_predictions == True).sum()